In [1]:
import poppler

In [4]:
doc = poppler.load_from_file("/data/Otherscape/Metro-Otherscape Core Book.pdf")

In [5]:
doc.infos()

{'Creator': 'Adobe InDesign 19.2 (Macintosh)',
 'Producer': 'Adobe PDF Library 17.0',
 'Trapped': '',
 'ModDate': datetime.datetime(2024, 7, 19, 15, 45, 24),
 'CreationDate': datetime.datetime(2024, 7, 19, 15, 41, 32)}

In [6]:
import pandas as pd

data = [
    (p, i, b.text, b.get_font_name(), b.get_font_size())
      for p in range(doc.pages)
      for i, b in enumerate(doc.create_page(p).text_list(doc.create_page(p).TextListOption.text_list_include_font))
]
df = pd.DataFrame(data, columns=["page", "word_in_page", "text", "font", "size"])
df

,page,word_in_page,text,font,size
0,0,0,Property,Helvetica,8.0
1,0,1,of,Helvetica,8.0
2,0,2,City,Helvetica,8.0
3,0,3,of,Helvetica,8.0
4,0,4,Mist.,Helvetica,8.0
...,...,...,...,...,...
196699,367,1039,ZipTie,YHFGWG+Isotonic-Rg,8.0
196700,367,1040,"Basilikum,",YHFGWG+Isotonic-Rg,8.0
196701,367,1041,zmef,YHFGWG+Isotonic-Rg,8.0
196702,367,1042,"(he/him),",YHFGWG+Isotonic-Rg,8.0


In [10]:
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 6.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.8 MB/s eta 0:00:00a 0:00:01


In [24]:
df["size"].unique()

array([  8.        ,  14.2083    ,  10.6562    ,  19.        ,
        10.        ,  23.6655    ,  14.        ,  10.5       ,
        17.        ,  41.        ,  60.        ,  41.40293406,
        16.        , 424.0026    ,  62.        ,  15.5       ,
        13.        ,   9.        ,  26.3769    ,   5.5       ,
        11.        ,  25.7238    ,  12.        ,  33.        ,
        11.2976    ,  22.1115    ,  30.        ,  37.        ,
        18.        ,  15.        ,  32.        ,   9.5       ,
        12.36726723,  21.01690086,  36.00003699,  36.        ,
        12.36734545,  21.00004021,  12.3673    ,  21.0169    ,
        20.99996811,  21.        ,   9.00002291,   8.99997124,
        12.36732611,  12.36728292,  20.99995603,  20.99999839,
         8.99998273,  12.36733337,  20.99999301,   9.999953  ,
        12.00003271,  12.00000202,   7.        ,  12.6       ,
        23.        ,  14.5       ,  19.9278    ,  13.2852    ,
         9.2996    ,  11.9567    ,  10.6282    ,  50.  

In [25]:
chapter_heading = []
section_heading = []
subsection_heading = []
current_doc = []

last_font_size = 0

docs = {}
def stringify(thing):
    return " ".join(thing)

for p in range(doc.pages):
    page = doc.create_page(p)
    for b in page.text_list(page.TextListOption.text_list_include_font):
        font_size = b.get_font_size()

        if font_size < last_font_size:
            current_doc.append("\n\n")

        if font_size < 20:
            if font_size > last_font_size:
                current_doc.append("\n")
            current_doc.append(b.text)

        if font_size > last_font_size:
            if font_size > 20:
                # end of section
                docs[f"{stringify(chapter_heading)}{(' - ' + stringify(section_heading)) if section_heading else ''}"] = f"{stringify(chapter_heading)}\n\n{stringify(section_heading)}\n{stringify(current_doc)}"
                current_doc = []
                section_heading = []
                if font_size > 30:
                    # end of chapter
                    chapter_heading = []
            elif font_size > 18:
                # end of sub-subsection
                docs[f"{stringify(chapter_heading)} - {stringify(section_heading)} - {stringify(subsection_heading)}"] = f"{stringify(chapter_heading)}\n\n{stringify(section_heading)}\n{stringify(subsection_heading)}\n{stringify(current_doc)}"
                current_doc = []
                subsection_heading = []            

        if font_size > 30 and font_size < 35:
            chapter_heading.append(b.text)

        if font_size > 20 and font_size < 30:
            section_heading.append(b.text)
        elif font_size > 18 and font_size < 20:
            subsection_heading.append(b.text)

        last_font_size = font_size
docs

{' -  - ': '\n\n\n\n\n Property of City of Mist. Issued to Tierra Duffield. Order #218284 \n CREDITS \n\n Creator, Lead Game Designer, and Creative Director: Amít Moshe Game Designers: Eran Aviram, Itamar Karbian, Kelly Black Developer: Kelly Black Editor: Eran Aviram Proofreaders: Marshall Oppel, Matthew Pook Game Producers: Omer Shapira, Inbar Barzilay Art Production: Erina Liddle Lead Writers: Colin "Mephit James" Wilson, Kelly Black, Tony Pi Writers: Amít Moshe, Eran Aviram, Kend\'or Wilson, Kevin Carpenter, Kristoff Alejo, Lexi Antoku, Dr. Yiftach Raphael Govreen Lead Artist: Isago Fukuda Additional Artists: Ario Murti , Bad Moon Studio, Daniel Liang, Derek Weselake, Jayden Morris, Krzysztof Bieniawski, Martina Belli, Misael "Gio" Manning, Sebastian Szmyd, Rasuli Iman Fajri Lead Graphic Designer: Manuel Serra Sáez Additional Layout: Alba Martínez Gil, Kelly Black, Kristoff Alejo Operations: Maria Jose Castañeda Customer Support: Alfonso Campos Social Media & Content: Kevin Carpent

In [ ]:
from utils.elastic import elastic_request
import requests
test = elastic_request(method=requests.post, url="_ingest/pipeline/clean_and_embed/_simulate", data={"docs":[{"_source": {"content": docs["Chapter 1: Step- By-Step Characters"]}}]})
test.json()

In [ ]:
def ingest(doc, title, index):
    cleaned_title = title.replace("?", "").replace("/", "-")
    rslt = elastic_request(method=requests.put,
                           url=f"{index}/_doc/{cleaned_title}?pipeline=clean_and_embed",
                           data={"content": doc})
    return rslt


In [ ]:
for title, item in docs.items():
    if title:
        try:
            rslt = ingest(item, title, "otherscape--metro-otherscape")
            rslt.raise_for_status()
        except Exception as e:
            print("Error: ", e)
            print(rslt.json())
            raise